In [1]:
import requests
import sqlite3
from time import sleep
from datetime import datetime, timedelta, date
from requests.exceptions import ConnectTimeout

# Configuración de la conexión a la base de datos SQLite
conn = sqlite3.connect('proyecto.db')
c = conn.cursor()

# Método para crear la tabla tokens en la base de datos
def create_tokens_table():
    c.execute('''CREATE TABLE IF NOT EXISTS tokens (
                    access_token TEXT,
                    refresh_token TEXT,
                    expiration_date TEXT
                )''')

# Verificar si la tabla tokens existe y crearla si no existe
create_tokens_table()

# Método para obtener el access_token y refresh_token de la base de datos
def get_tokens():
    c.execute("SELECT * FROM tokens")
    tokens = c.fetchone()
    if tokens is None:
        return None
    access_token, refresh_token, expiration_date = tokens
    expiration_date = datetime.strptime(expiration_date, '%Y-%m-%d %H:%M:%S')
    return access_token, refresh_token, expiration_date

# Método para guardar el access_token y refresh_token en la base de datos
def save_tokens(access_token, refresh_token, expiration_date):
    c.execute("DELETE FROM tokens")
    c.execute("INSERT INTO tokens VALUES (?, ?, ?)",
            (access_token, refresh_token, expiration_date))
    conn.commit()

# Método para obtener un nuevo access_token y refresh_token
def get_new_tokens(refresh_token):
    url = "https://api.mercadolibre.com/oauth/token"
    payload = {
        "grant_type": "refresh_token",
        "client_id": "5030313997317379",
        "client_secret": "zTJax3dLAiog35gQdaOVEhTSwxXxbTTY",
        "refresh_token": refresh_token
    }
    response = requests.post(url, data=payload)
    response.raise_for_status()
    data = response.json()
    access_token = data['access_token']
    refresh_token = data['refresh_token']
    expires_in = data['expires_in']
    expiration_date = datetime.now() + timedelta(seconds=expires_in)
    expiration_date_str = expiration_date.strftime('%Y-%m-%d %H:%M:%S')
    save_tokens(access_token, refresh_token, expiration_date_str)
    return access_token, refresh_token, expiration_date_str

# Método para hacer la petición con el access_token
def make_request(access_token, url):
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data

# Método para hacer una petición a la API
def do_request(url, params=None):
    tokens = get_tokens()
    if tokens is None or tokens[2] < datetime.now():
        refresh_token = tokens[1] if tokens is not None else input("Ingrese refresh_token: ")
        access_token, refresh_token, expiration_date = get_new_tokens(refresh_token)
    else:
        access_token, refresh_token, expiration_date = tokens
    
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    retries = 3  # Número máximo de reintentos
    retry_delay = 30  # Tiempo de espera en segundos antes de cada reintento
    
    for i in range(retries):
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)  # Establece un tiempo de espera de 10 segundos
            response.raise_for_status()
            data = response.json()
            return data
        except ConnectTimeout:
            print("Error de tiempo de espera de conexión. Se realizará un reintento después de 30 segundos...")
            sleep(retry_delay)
    
    print("Se ha excedido el número máximo de intentos de conexión. Se aborta la solicitud.")
    return None

# Método para obtener las visitas de cada producto
def get_product_visits(product_id):
    today = date.today()
    last_month = today - timedelta(days=30)
    
    endpoint = f"https://api.mercadolibre.com/visits/items"
    params = {
        "ids": product_id,
        "date_from": last_month.isoformat(),
        "date_to": today.isoformat()
    }

    # Aquí se utiliza la función do_request para realizar la solicitud con el access token
    response = do_request(endpoint, params=params)
    if response and isinstance(response, dict):
        product_visits = response.get(product_id)
        return product_visits
    
    return None



url = "https://api.mercadolibre.com/trends/MLC"

productos = {
    "tablets": "https://api.mercadolibre.com/sites/MLC/search?category=MLC82067",
    "notebooks": "https://api.mercadolibre.com/sites/MLC/search?category=MLC1652",
    "pcs": "https://api.mercadolibre.com/sites/MLC/search?category=MLC181025",
    "impresoras": "https://api.mercadolibre.com/sites/MLC/search?category=MLC1676"
}
# datos json
datos_json = do_request(productos["tablets"])['results']

In [2]:
datos_json

[{'id': 'MLC933990141',
  'title': 'Apple iPad (9ª Generación) 10.2  Wi-fi 64gb - Color Plata',
  'condition': 'new',
  'thumbnail_id': '907886-MLU69497668167_052023',
  'catalog_product_id': 'MLC18498454',
  'listing_type_id': 'gold_pro',
  'permalink': 'https://www.mercadolibre.cl/apple-ipad-9-generacion-102-wi-fi-64gb-color-plata/p/MLC18498454',
  'buying_mode': 'buy_it_now',
  'site_id': 'MLC',
  'category_id': 'MLC82067',
  'domain_id': 'MLC-TABLETS',
  'thumbnail': 'http://http2.mlstatic.com/D_907886-MLU69497668167_052023-I.jpg',
  'currency_id': 'CLP',
  'order_backend': 1,
  'price': 329990,
  'original_price': 372990,
  'sale_price': None,
  'sold_quantity': 500,
  'available_quantity': 500,
  'official_store_id': 1078,
  'official_store_name': 'Apple',
  'use_thumbnail_id': True,
  'accepts_mercadopago': True,
  'tags': ['extended_warranty_eligible',
   'good_quality_picture',
   'good_quality_thumbnail',
   'immediate_payment',
   'cart_eligible',
   'best_seller_candidate',

In [27]:
%%time

import pandas as pd
import json
from time import sleep

# Método para crear la tabla en la base de datos
def create_table(table_name):
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
                    id TEXT,
                    title TEXT,
                    condition TEXT,
                    price INTEGER,
                    permalink TEXT,
                    thumbnail TEXT,
                    sold_quantity INTEGER,
                    available_quantity INTEGER,
                    seller_id TEXT,
                    seller_nickname TEXT,
                    brand TEXT,
                    line TEXT,
                    model TEXT,
                    shipping INTEGER,
                    visits_last_month INTEGER,
                    date_retrieved TEXT
                )''')
    
# Método para insertar los datos en la tabla
def insert_data(table_name, data):
    columns = ', '.join(data[0].keys())
    placeholders = ', '.join([':' + key for key in data[0].keys()])
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    c.executemany(query, data)
    conn.commit()

# Método para obtener los datos de la tabla correspondientes a esta semana
def get_data(table_name):
    # Obtener la fecha de inicio y fin de la semana actual
    fecha_actual = datetime.now()
    fecha_inicio_semana = fecha_actual - timedelta(days=6)

    # Convertir las fechas a strings
    fecha_inicio_semana_str = fecha_inicio_semana.strftime('%Y-%m-%d %H:%M:%S')
    fecha_fin_semana_str = fecha_actual.strftime('%Y-%m-%d %H:%M:%S')

    # Consulta SQL para obtener los datos de la semana actual
    query = f"SELECT * FROM {table_name} WHERE date_retrieved BETWEEN ? AND ?"
    c.execute(query, (fecha_inicio_semana_str, fecha_fin_semana_str))

    # Obtener los nombres de las columnas
    column_names = [description[0] for description in c.description]

    # Obtener los datos de la consulta
    rows = c.fetchall()

    # Crear el DataFrame con los datos y los nombres de las columnas
    df = pd.DataFrame(rows, columns=column_names)

    return df

# Método para verificar si los datos de la API son más viejos a una semana
def check_data_age(table_name):
    c.execute(f"SELECT MAX(date_retrieved) FROM {table_name}")
    retrieved_date = c.fetchone()[0]
    if retrieved_date is None:
        return True
    else:
        retrieved_date = datetime.strptime(retrieved_date, '%Y-%m-%d %H:%M:%S')
        return datetime.now() - retrieved_date > timedelta(days=6)


# ...

def obtener_datos_por_pagina(total_resultados, url, table_name):
    resultados_por_pagina = 50
    paginas = total_resultados // resultados_por_pagina

    datos_filtrados_full = []

    for pagina in range(1, paginas + 1):
        parametros = {
            "offset": (pagina - 1) * resultados_por_pagina,
            "limit": resultados_por_pagina
        }
        respuesta = do_request(url, parametros)
        if "results" in respuesta and respuesta["results"]:
            datos_pagina = respuesta["results"]
        else:
            break

        for publicacion in datos_pagina:
            publicacion_filtrada = {
                "id": publicacion["id"],
                "title": publicacion["title"],
                "condition": publicacion["condition"],
                "price": publicacion["price"],
                "permalink": publicacion["permalink"],
                "thumbnail": publicacion["thumbnail"],
                "sold_quantity": publicacion["sold_quantity"],
                "available_quantity": publicacion["available_quantity"],
                "seller_id": publicacion["seller"]["id"],
                "seller_nickname": publicacion["seller"]["nickname"],
                "brand": None,
                "line": None,
                "model": None,
                "shipping": publicacion["shipping"]["free_shipping"],
                "visits_last_month": get_product_visits(publicacion["id"]),
                "date_retrieved": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            for atributo in publicacion["attributes"]:
                if atributo["id"] == "BRAND":
                    publicacion_filtrada["brand"] = atributo["value_name"]
                elif atributo["id"] == "LINE":
                    publicacion_filtrada["line"] = atributo["value_name"]
                elif atributo["id"] == "MODEL":
                    publicacion_filtrada["model"] = atributo["value_name"]

            datos_filtrados_full.append(publicacion_filtrada)

    # Inserta los datos en la tabla
    insert_data(table_name, datos_filtrados_full)

    dataframe = pd.DataFrame(datos_filtrados_full)

    return dataframe


total_resultados = 3950

# Ejecuta la función y obtiene los datos filtrados por página si es necesario
def obtener_data(producto):
    create_table(producto)
    if check_data_age(producto):
        df_product = obtener_datos_por_pagina(total_resultados, productos[producto], producto)
    else:
        # Obtiene los datos de la tabla
        df_product = get_data(producto)
    return df_product


# Llama a la función obtener_data con el producto "notebooks"
df_notebooks = obtener_data("notebooks")
df_tablets = obtener_data("tablets")
df_pcs = obtener_data("pcs")
df_impresoras = obtener_data("impresoras")
df_notebooks

CPU times: total: 109 ms
Wall time: 101 ms


,id,title,condition,price,permalink,thumbnail,sold_quantity,available_quantity,seller_id,seller_nickname,brand,line,model,shipping,visits_last_month,date_retrieved
0,MLC1791436106,Notebook Asus Vivobook Pro 15 Oled Amd Ryzen 5...,new,779990,https://www.mercadolibre.cl/notebook-asus-vivo...,http://http2.mlstatic.com/D_755031-MLU70371398...,5,1,1020208804,ANOVO ANDES,Asus,Vivobook Pro 15 OLED,M6500QC-L1015W,1,542,2023-07-25 08:03:03
1,MLC1591946838,"Notebook Hp 240 G9 Plateada 14 , Intel Celeron...",new,217000,https://www.mercadolibre.cl/notebook-hp-240-g9...,http://http2.mlstatic.com/D_809147-MLA52105513...,150,1,791863127,OPORTUTEK-CHILE,HP,None,240 G9,1,5021,2023-07-25 08:03:03
2,MLC979733842,"Macbook Air M1 2020 Gris Espacial 13.3 , Apple...",new,722977,https://www.mercadolibre.cl/macbook-air-m1-202...,http://http2.mlstatic.com/D_614885-MLA49588273...,500,50,550063615,MERCADOLIBRE ELECTRONICA_CL,Apple,MacBook Air,M1 2020,1,122330,2023-07-25 08:03:03
3,MLC1131351756,"Macbook Air M1 2020 Plata 13.3 , Apple M1 8gb...",new,718495,https://www.mercadolibre.cl/macbook-air-m1-202...,http://http2.mlstatic.com/D_927163-MLA48622311...,250,50,550063615,MERCADOLIBRE ELECTRONICA_CL,Apple,MacBook Air,M1 2020,1,34598,2023-07-25 08:03:04
4,MLC1171990179,"Macbook Air M1 2020 Oro 13.3 , Apple M1 8gb D...",new,722977,https://www.mercadolibre.cl/macbook-air-m1-202...,http://http2.mlstatic.com/D_868622-MLA48622311...,150,50,550063615,MERCADOLIBRE ELECTRONICA_CL,Apple,MacBook Air,M1 2020,1,9529,2023-07-25 08:03:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3929,MLC1401267901,"Ordenador Portátil De Doble Banda De 15,6 Pulg...",new,324811,https://articulo.mercadolibre.cl/MLC-140126790...,http://http2.mlstatic.com/D_718506-CBT53217217...,0,50,1143863887,CNTEMPLEG-CLR,Generic,Unique,OEM,1,0,2023-07-25 08:23:24
3930,MLC1401278111,Notebook Portátil De 14 Pulgadas Para Intel J4...,new,221464,https://articulo.mercadolibre.cl/MLC-140127811...,http://http2.mlstatic.com/D_685767-CBT70414944...,0,50,1143863887,CNTEMPLEG-CLR,Generic,None,OEM,1,0,2023-07-25 08:23:25
3931,MLC1403245103,Ordenador Portátil De 14.1 Pulgadas Fhd Ultra ...,new,219513,https://articulo.mercadolibre.cl/MLC-140324510...,http://http2.mlstatic.com/D_876231-CBT70500135...,0,50,1143863887,CNTEMPLEG-CLR,Generic,None,OEM,1,0,2023-07-25 08:23:25
3932,MLC1404610319,"Notebook Fhd De 14.1 Pulgadas, Ultra Delgada, ...",new,228596,https://articulo.mercadolibre.cl/MLC-140461031...,http://http2.mlstatic.com/D_905611-CBT70614703...,0,50,1143863887,CNTEMPLEG-CLR,Generic,Ultra,OEM,1,0,2023-07-25 08:23:25


In [26]:
datetime.now().date()

datetime.date(2023, 7, 25)

In [28]:
# Definir una función para agregar la columna 'producto'
def agregar_producto(df):
    df['producto'] = df['brand'] + ' '+ df['line'] + ' ' + df['model']
    return df

# Iterar sobre los DataFrames y agregar la columna 'producto' a cada uno de ellos
dataframes = [df_notebooks, df_tablets, df_pcs, df_impresoras]
nombres = ["notebooks", "tablets", "pcs", "impresoras"]

for i in range(len(dataframes)):
    dataframes[i] = agregar_producto(dataframes[i])
    

In [29]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit

def calcular_metricas_producto(df):
    
    # Filtrar los resultados con menos de 50 visitas
    df = df[df['visits_last_month'] >= 50]
    
    # Agrupar por la columna "producto"
    grouped_df = df.groupby('producto')
    
    # Calcular la suma de "sold_quantity" y "visits_last_month" para cada grupo
    total_sold_quantity = grouped_df['sold_quantity'].sum()
    total_visits_last_month = grouped_df['visits_last_month'].sum()
    
    # Calcular el promedio de "price" para cada grupo
    average_price = grouped_df['price'].mean()
    
    # Obtener el "permalink" y "thumbnail" del primer producto detectado en cada grupo
    first_product_data = grouped_df.first()
    permalink = first_product_data['permalink']
    thumbnail = first_product_data['thumbnail']
    
    # Obtener el primer valor de "brand", "model" y "line" para cada grupo
    brand = first_product_data['brand']
    model = first_product_data['model']
    line = first_product_data['line']
    
    # Contar el número único de vendedores para cada grupo (producto)
    total_sellers = grouped_df['seller_id'].nunique()
    
     # Calcular la variable "sells_per_visit" (ventas por visita)
    sells_per_visit = total_sold_quantity / total_visits_last_month
    sells_per_visit = sells_per_visit.fillna(0)  # Reemplazar NaN con cero
    
    # Calcular la desviación estándar de "visits_last_month" para cada grupo (producto)
    std_visits_last_month = grouped_df['visits_last_month'].std()
    
    # Reemplazar NaN en std_visits_last_month con el promedio de las desviaciones estándar existentes
    std_visits_last_month_mean = std_visits_last_month.mean()
    std_visits_last_month = std_visits_last_month.fillna(std_visits_last_month_mean)
    
    # Crear el nuevo DataFrame con las métricas calculadas
    result_df = pd.DataFrame({
        'product': total_sold_quantity.index,
        'brand': brand.values,
        'line': line.values,
        'model': model.values,
        'average_price': average_price.values,
        'total_sold_quantity': total_sold_quantity.values,
        'total_visits_last_month': total_visits_last_month.values,
        'total_sellers': total_sellers.values,
        'sells_per_visit': sells_per_visit.values,
        'std_visits_last_month': std_visits_last_month.values,
        'permalink': permalink.values,
        'thumbnail': thumbnail.values
        
    })
    
    # Ordenar el DataFrame por "total_visits_last_month" en orden descendente
    #result_df = result_df.sort_values(by='total_visits_last_month', ascending=False)
    result_df = result_df.sort_values(by='total_sellers', ascending=False)
    #result_df = result_df.sort_values(by='total_sold_quantity', ascending=False)

    
    return result_df

df_notebooks.info()

df_notebooks_pp = calcular_metricas_producto(df_notebooks)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df_notebooks_pp, df_notebooks_pp["total_visits_last_month"]):
    strat_train_set = df_notebooks_pp.loc[train_index]
    strat_test_set = housing.loc[test_index]
df_notebooks_pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3934 entries, 0 to 3933
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  3934 non-null   object
 1   title               3934 non-null   object
 2   condition           3934 non-null   object
 3   price               3934 non-null   int64 
 4   permalink           3934 non-null   object
 5   thumbnail           3934 non-null   object
 6   sold_quantity       3934 non-null   int64 
 7   available_quantity  3934 non-null   int64 
 8   seller_id           3934 non-null   object
 9   seller_nickname     3934 non-null   object
 10  brand               3930 non-null   object
 11  line                3373 non-null   object
 12  model               3842 non-null   object
 13  shipping            3934 non-null   int64 
 14  visits_last_month   3934 non-null   int64 
 15  date_retrieved      3934 non-null   object
 16  producto            3329

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Definir una clase personalizada para calcular la variable objetivo
class ObjectiveValueCalculator(BaseEstimator, TransformerMixin):
    def __init__(self, weight_total_visits=0.6, weight_sells_per_visit=0.1, weight_std_visits=0.3):
        self.weight_total_visits = weight_total_visits
        self.weight_sells_per_visit = weight_sells_per_visit
        self.weight_std_visits = weight_std_visits

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_result = X.copy()
        X_result['label'] = (self.weight_total_visits * X['total_visits_last_month']) + \
                            (self.weight_sells_per_visit * X['sells_per_visit']) + \
                            (self.weight_std_visits * (1 - X['std_visits_last_month']))
        return X_result

# Definir las columnas numéricas y categóricas para el ColumnTransformer
numeric_features = ['average_price', 'total_sold_quantity', 'total_visits_last_month', 'total_sellers', 'sells_per_visit', 'std_visits_last_month']
categorical_features = ['brand', 'model']

# Definir la Pipeline para la transformación de datos
preprocessing_pipeline = ColumnTransformer([
    ('numeric', MinMaxScaler(), numeric_features),
    ('categorical', OneHotEncoder(sparse_output=False), categorical_features)
])

class ObjectiveValueCalculator(BaseEstimator, TransformerMixin):
    def __init__(self, weight_total_visits=0.6, weight_sells_per_visit=0.1, weight_std_visits=0.3):
        self.weight_total_visits = weight_total_visits
        self.weight_sells_per_visit = weight_sells_per_visit
        self.weight_std_visits = weight_std_visits
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        # Ajustar el MinMaxScaler a los datos
        self.scaler.fit(X)
        return self

    def transform(self, X):
        # Escalar los datos utilizando el MinMaxScaler
        X_scaled = self.scaler.transform(X)

        # Calcular la variable objetivo
        label = (self.weight_total_visits * X_scaled[:, 2]) + \
                (self.weight_sells_per_visit * X_scaled[:, 4]) + \
                (self.weight_std_visits * (1 - X_scaled[:, 5]))

        return label

# Suponiendo que ya tienes un DataFrame df_notebooks con las columnas 'producto' y 'permalink'
# Calculamos las métricas y obtenemos el nuevo DataFrame con los resultados
df_notebooks_mp = preprocessing_pipeline.fit_transform(df_notebooks_pp)

# Definir el objetivo para el modelo de Machine Learning utilizando la clase personalizada ObjectiveValueCalculator
objective_calculator = ObjectiveValueCalculator()
df_notebooks_final = objective_calculator.fit_transform(df_notebooks_mp)

df_notebooks_mp

df_notebooks_final.shape

(1435,)

In [58]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


lin_reg = LinearRegression()
lin_reg.fit(df_notebooks_mp, df_notebooks_final)

# Hacer predicciones en el conjunto de datos df_notebooks_mp
predicciones = lin_reg.predict(df_notebooks_mp)

predicciones.shape

# Calculo del error
lin_mae = mean_absolute_error(df_notebooks_final, predicciones)
lin_mae

3.0645250177980803e-16